# Cache

A common way to optimize applications is by using caching, which involves storing the outputs of certain processes for a set period. For FastAPI, the [`fastapi_cache`](https://github.com/long2ice/fastapi-cache) library provides a convenient way to implement caching, simplifying the process significantly.

In [1]:
import requests

## Setup

To run the examples associated with `fastapi_cache`, we need a specific setup: an API and a caching backend. The following Docker Compose file defines both an API service and a Redis service to be used as the caching backend.

In [2]:
%%writefile cache_files/docker-compose.yml
services:
  fast_api:
    image: fastapi_experiment
    container_name: cache_example_api
    volumes:
      - ./app.py:/app.py
    command: /bin/sh -c "\
      pip3 install redis fastapi-cache2[redis] && \
      uvicorn --host 0.0.0.0 --reload app:app"
    network_mode: host
    stdin_open: true
    tty: true
  redis:
    image: redis:7.4.0
    container_name: cache_example_redis
    network_mode: host
    command: redis-server --port 6380

Overwriting cache_files/docker-compose.yml


The following cell starts the application. 

In [3]:
%%bash
cd cache_files/
docker compose up -d &> /dev/null

Don't forget to clean the environment after all.

In [112]:
%%bash
cd cache_files/
docker compose down &> /dev/null

## Check cache

Caching isn't magic; you can find your cached records in the backend you're using.

---

The following example defines a program that returns a random value each time you access the endpoint, requiring you to specify a `user_id`. 

In [4]:
%%writefile cache_files/app.py
from random import random

from fastapi import FastAPI

from fastapi_cache import FastAPICache
from fastapi_cache.decorator import cache
from fastapi_cache.backends.redis import RedisBackend

from redis import asyncio as aioredis

redis = aioredis.from_url("redis://localhost:6380")
FastAPICache.init(RedisBackend(redis), prefix="fastapi-cache")

app = FastAPI()

@app.get("/{id}")
@cache(expire=600)
def index(id: int):
    return random()

Overwriting cache_files/app.py


Now you can try making requests to the application.  Different URLs will return different values. 

In [5]:
%%bash
curl -s localhost:8000/10
echo
curl -s localhost:8000/20

0.23241575866987674
0.771544600406915

However, requests to the same URL will return the same values as before, because the results have been cached from previous calls.

In [6]:
%%bash
curl -s localhost:8000/10
echo
curl -s localhost:8000/20

0.23241575866987674
0.771544600406915

Let's check if the cached values are present in the Redis database.

In [7]:
!redis-cli -h localhost -p 6380 keys '*'

1) "fastapi-cache::c2ab82769e6470b9ef90fa0de4fbb612"
2) "fastapi-cache::5511deb9b96a93779726aed3f9e4d8bc"


We see two keys that begin with the string "fastapi-cache", just as we specified in the `prefix` argument. The following Python code prints the values of all keys available in the Redis database. 

In [8]:
import redis

r = redis.Redis(host='localhost', port=6380, db=0)

matching_keys = []
cursor = '0'
while cursor != 0:
    cursor, keys = r.scan(cursor=cursor, match='fastapi-cache*')
    matching_keys.extend(keys)

if matching_keys:
    values = r.mget(matching_keys)
    for key, value in zip(matching_keys, values):
        print(f"{key.decode('utf-8')}: {value.decode('utf-8')}")

fastapi-cache::c2ab82769e6470b9ef90fa0de4fbb612: 0.771544600406915
fastapi-cache::5511deb9b96a93779726aed3f9e4d8bc: 0.23241575866987674


The values retrieved from Redis match the values we received from the API responses.

## Query params

You can also use query parameters with `fastapi_cache`. Each unique combination of query parameters will have its own cached variable.

---

The following API defines an endpoint that uses query parameters.

In [9]:
%%writefile cache_files/app.py
from random import random

from fastapi import FastAPI

from fastapi_cache import FastAPICache
from fastapi_cache.decorator import cache
from fastapi_cache.backends.redis import RedisBackend

from redis import asyncio as aioredis

redis = aioredis.from_url("redis://localhost:6380")
FastAPICache.init(RedisBackend(redis), prefix="fastapi-cache")

app = FastAPI()

@app.get("/")
@cache(expire=600)
def index(id: int):
    return random()

Overwriting cache_files/app.py


Now let's try passing different query parameters to the API.

In [10]:
%%bash
curl -s localhost:8000/?id=50
echo
curl -s localhost:8000/?id=60
echo
echo
curl -s localhost:8000/?id=50
echo
curl -s localhost:8000/?id=60

0.8803390858597414
0.7743148886889138

0.8803390858597414
0.7743148886889138

Different query parameters will produce different results. However, identical sets of query parameters will return the same values due to caching.

## Key builder

A special function that generates keys to be used in Redis for caching. You can get default `key_builder` by using `FastAPICache.get_key_builder` method of the initialised `FastAPICache`.

---

The following cell demonstrates how to retrieve the default `key_builder` and even calls it with an incomplete set of arguments.

In [11]:
from fastapi_cache import FastAPICache
from fastapi_cache.backends.redis import RedisBackend
from redis import asyncio as aioredis

redis = aioredis.from_url("redis://localhost:6380")
FastAPICache.init(RedisBackend(redis), prefix="fastapi-cache")

key_builder = FastAPICache.get_key_builder()
def test_function():
    pass
key_builder(func=test_function, args=[], kwargs={})

':f0a9508ba2bac36e3742d56a5c0859cb'

As a result, a random hash is generated.

## Exceptions caching

`fastapi_cache` does not provide caching of exceptions. Therefore, if your code throws an exception, each subsequent call to the same endpoint will re-execute all code leading up to the exception.

---

The following code provides an application that throws an exception every time it's called. The exception message is generated randomly.

In [12]:
%%writefile cache_files/app.py
from random import random

from fastapi import FastAPI
from fastapi.exceptions import HTTPException

from fastapi_cache import FastAPICache
from fastapi_cache.decorator import cache
from fastapi_cache.backends.redis import RedisBackend

from redis import asyncio as aioredis

redis = aioredis.from_url("redis://localhost:6380")
FastAPICache.init(RedisBackend(redis), prefix="fastapi-cache")

app = FastAPI()

@app.get("/")
@cache(expire=600)
def index():
    raise HTTPException(500, str(random()))

Overwriting cache_files/app.py


Each call to the API results in an error with a different message.

In [13]:
%%bash
curl -s localhost:8000/
echo
curl -s localhost:8000/

{"detail":"0.6015026643515221"}
{"detail":"0.971603006025925"}

### JSONResponse as exception

As possible solution you can reproduce outputs of the `fastapi.exceptions.HTTPException` using `fastapi.responses.JSONResponse`.

---

The following cell demonstrates the creation of an application that generates a `JSONResponse` with a `status_code=500` and a random message under the `detail` key for each response.

In [14]:
%%writefile cache_files/app.py
from random import random

from fastapi import FastAPI
from fastapi.responses import JSONResponse

from fastapi_cache import FastAPICache
from fastapi_cache.decorator import cache
from fastapi_cache.backends.redis import RedisBackend

from redis import asyncio as aioredis

redis = aioredis.from_url("redis://localhost:6380")
FastAPICache.init(RedisBackend(redis), prefix="fastapi-cache")

app = FastAPI()

@app.get("/")
@cache(expire=600)
def index():
    data = {"detail": str(random())}
    return JSONResponse(content=data, status_code=500)

Overwriting cache_files/app.py


Now let's make some requests to the created application.

In [15]:
response = requests.get("http://localhost:8000")

print("Status code", response.status_code)
print("Content", response.content)
print("Headers", response.headers)

Status code 500
Content b'{"detail":"0.5846017203682882"}'
Headers {'date': 'Thu, 22 Aug 2024 14:15:03 GMT', 'server': 'uvicorn', 'content-length': '31', 'content-type': 'application/json'}


The next cell repeats the same request - so it would be new value.

In [16]:
response = requests.get("http://localhost:8000")

print("Status code", response.status_code)
print("Content", response.content)
print("Headers", response.headers)

Status code 200
Content b'{"detail":"0.5846017203682882"}'
Headers {'date': 'Thu, 22 Aug 2024 14:15:06 GMT', 'server': 'uvicorn', 'content-length': '31', 'content-type': 'application/json', 'cache-control': 'max-age=597', 'etag': 'W/4408960420008572764', 'x-fastapi-cache': 'HIT'}


While they formally have the same structure as responses from `HTTPException`, they were actually cached.